In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data collection and preprocessing

In [ ]:
movies = pd.read_csv('/content/movies.csv')

In [ ]:
movies.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
movies.shape

(4803, 24)

In [ ]:
#selecting the relevant features for recommendations
selected_features = ['genres','keywords','title','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'title', 'tagline', 'cast', 'director']


In [ ]:
#replacing the null values with null string
for feature in selected_features:
  movies[feature] = movies[feature].fillna('')

In [ ]:
movies.tail()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,,http://shanghaicalling.com/,126186,,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,"[{'name': 'Daniel Hsia', 'gender': 2, 'departm...",Daniel Hsia
4802,4802,0,Documentary,NaN,25975,obsession camcorder crush dream girl,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,,My Date with Drew,6.3,16,Drew Barrymore Brian Herzlinger Corey Feldman ...,"[{'name': 'Clark Peterson', 'gender': 2, 'depa...",Brian Herzlinger


In [ ]:
#combining all the 5 selected features

combined_features = movies['genres']+' '+movies['keywords']+' '+movies['tagline']+' '+movies['title']+' '+movies['cast']+' '+movies['director']

In [ ]:
combined_features.head()

,0
0,Action Adventure Fantasy Science Fiction cultu...
1,Adventure Fantasy Action ocean drug abuse exot...
2,Action Adventure Crime spy based on novel secr...
3,Action Crime Drama Thriller dc comics crime fi...
4,Action Adventure Science Fiction based on nove...


In [ ]:
#converting test data to feature vectors
vectorizer = TfidfVectorizer()

In [ ]:
feature_vec = vectorizer.fit_transform(combined_features)

In [ ]:
print(feature_vec)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 134944 stored elements and shape (4803, 19184)>
  Coords	Values
  (0, 243)	0.07630361845708403
  (0, 322)	0.08752209522589645
  (0, 5882)	0.10791658457595885
  (0, 15011)	0.10068472429000817
  (0, 6055)	0.10056743537917023
  (0, 4103)	0.2078190465468255
  (0, 3408)	0.21574818782392277
  (0, 6519)	0.15884357175977007
  (0, 15901)	0.3255851082321633
  (0, 18385)	0.1206723617514615
  (0, 3587)	0.2424810120582838
  (0, 15785)	0.20449485056097091
  (0, 5519)	0.22061174669983705
  (0, 16904)	0.05365726945306952
  (0, 18835)	0.12179929157015999
  (0, 12356)	0.07571236182305312
  (0, 12700)	0.2552737122112953
  (0, 1183)	0.2771429775697421
  (0, 14749)	0.14568185359096344
  (0, 18845)	0.22968831190527225
  (0, 19148)	0.19502634639381394
  (0, 14718)	0.21153518149440187
  (0, 15542)	0.20008526614580363
  (0, 18477)	0.1927717674394528
  (0, 16170)	0.1471845509560594
  :	:
  (4801, 2698)	0.2411630217643225
  (4801, 14565)	0.25682086501

cosine similarity


In [ ]:
#using cosine similarity
sim = cosine_similarity(feature_vec)

In [ ]:
print(sim)

[[1.         0.07294698 0.03533251 ... 0.         0.         0.        ]
 [0.07294698 1.         0.02792771 ... 0.04419983 0.         0.        ]
 [0.03533251 0.02792771 1.         ... 0.         0.04636139 0.        ]
 ...
 [0.         0.04419983 0.         ... 1.         0.         0.05551043]
 [0.         0.         0.04636139 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.05551043 0.         1.        ]]


In [ ]:
#getting movie name from user
movie_name = input("Enter your favourite name : ")

Enter your favourite name : iron man


In [ ]:
#creating a list with all teh movies given in the data set
list_of_all_titles = movies['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [ ]:
#finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [ ]:
close_match = find_close_match[0]

In [ ]:
print(close_match)

Iron Man


In [ ]:
#finding the index of the movie with title
index_of_movie = movies[movies.title == close_match]['index'].values[0]
print(index_of_movie)

68


In [ ]:
#getting a list of similar movies
sim_score = list(enumerate(sim[index_of_movie]))
print(sim_score)

[(0, np.float64(0.03178870695294604)), (1, np.float64(0.04655013142047511)), (2, np.float64(0.012934561590593172)), (3, np.float64(0.0057966667395479175)), (4, np.float64(0.03165067765535343)), (5, np.float64(0.02835709115641611)), (6, np.float64(0.07168720055110567)), (7, np.float64(0.2087500626202012)), (8, np.float64(0.0069753104824515)), (9, np.float64(0.0660538678079144)), (10, np.float64(0.0686780200727214)), (11, np.float64(0.011033952554608314)), (12, np.float64(0.02568337857697453)), (13, np.float64(0.011045653769360248)), (14, np.float64(0.1190726609068993)), (15, np.float64(0.006420912675943166)), (16, np.float64(0.21440814789466553)), (17, np.float64(0.011514125845904494)), (18, np.float64(0.03912324028978291)), (19, np.float64(0.06954741615159196)), (20, np.float64(0.08802397045896385)), (21, np.float64(0.010414140704364687)), (22, np.float64(0.005643218539367042)), (23, np.float64(0.005586000071139674)), (24, np.float64(0.011990821940353034)), (25, np.float64(0.0)), (26, 

In [ ]:
#sorting movies based on their similarity score
sorted_sim = sorted(sim_score,key=lambda x:x[1],reverse=True)
print(sorted_sim)

[(68, np.float64(1.0000000000000002)), (79, np.float64(0.45614934334351387)), (31, np.float64(0.35885416360040373)), (16, np.float64(0.21440814789466553)), (7, np.float64(0.2087500626202012)), (182, np.float64(0.19790382074393092)), (26, np.float64(0.18481877406697328)), (85, np.float64(0.1829621359119527)), (511, np.float64(0.16140767377903908)), (3623, np.float64(0.15382104995893334)), (64, np.float64(0.14449608709036715)), (203, np.float64(0.14087010935407607)), (174, np.float64(0.1383148012125137)), (101, np.float64(0.1344393143974127)), (788, np.float64(0.1265975877268307)), (1740, np.float64(0.1255223313358453)), (46, np.float64(0.1253169378798418)), (94, np.float64(0.12429106316337901)), (126, np.float64(0.12391547678033554)), (169, np.float64(0.12370815738198507)), (131, np.float64(0.12364422935328384)), (14, np.float64(0.1190726609068993)), (33, np.float64(0.11885828396636185)), (38, np.float64(0.11834263945039614)), (2487, np.float64(0.11648585931776062)), (783, np.float64(0.

In [ ]:
#print the name of similar movies based on index
print('Movies suggested for you -')
i=1
for m in sorted_sim:
  idx=m[0]
  title_from_idx = movies[movies.index==idx]['title'].values[0]
  if (i<=30): # recommend as many as required
    print(i,'.',title_from_idx)
    i+=1

Movies suggested for you -
1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . The Avengers
5 . Avengers: Age of Ultron
6 . Ant-Man
7 . Captain America: Civil War
8 . Captain America: The Winter Soldier
9 . X-Men
10 . Made
11 . X-Men: Apocalypse
12 . X2
13 . The Incredible Hulk
14 . X-Men: First Class
15 . Deadpool
16 . Kick-Ass 2
17 . X-Men: Days of Future Past
18 . Guardians of the Galaxy
19 . Thor: The Dark World
20 . Captain America: The First Avenger
21 . G-Force
22 . Man of Steel
23 . X-Men: The Last Stand
24 . The Amazing Spider-Man 2
25 . Duets
26 . Mortdecai
27 . The Last Airbender
28 . Zathura: A Space Adventure
29 . The Good Night
30 . The Best Man


Movie Recommendation System


In [ ]:
#getting movie name from user
movie_name = input("Enter your favourite name : ")

list_of_all_titles = movies['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)

close_match = find_close_match[0]

index_of_movie = movies[movies.title == close_match]['index'].values[0]

sim_score = list(enumerate(sim[index_of_movie]))

sorted_sim = sorted(sim_score,key=lambda x:x[1],reverse=True)

print('Movies suggested for you -')

i=1
for m in sorted_sim:
  idx=m[0]
  title_from_idx = movies[movies.index==idx]['title'].values[0]
  if (i<=30): # recommend as many as required
    print(i,'.',title_from_idx)
    i+=1

Enter your favourite name : red
Movies suggested for you -
1 . Dredd
2 . The Wackness
3 . Vantage Point
4 . Star Trek Beyond
5 . Riddick
6 . Star Trek Into Darkness
7 . The Purge
8 . Black Water Transit
9 . Pete's Dragon
10 . The Darkest Hour
11 . Priest
12 . Zipper
13 . The Last Days on Mars
14 . Metropolis
15 . St. Trinian's
16 . District 9
17 . The Chronicles of Riddick
18 . Beavis and Butt-Head Do America
19 . Snow Angels
20 . The Killer Inside Me
21 . 300
22 . Idiocracy
23 . Aberdeen
24 . The Bourne Supremacy
25 . The Cave
26 . Imagine Me & You
27 . Walking With Dinosaurs
28 . RED
29 . Next Day Air
30 . The Adventurer: The Curse of the Midas Box
